## Demonstrate evaluating the surrogate model and unit transformations
This model was trained by Auralee to predict beam properties at OTR2 using injector
PVs for LCLS.

In [1]:
import json
import torch
import pprint

model_info = json.load(open("configs/model_info.json"))
pv_info = json.load(open("configs/pv_info.json"))

pprint.pprint(model_info)
pprint.pprint(pv_info)


{'loc_in': {'CQ01:b1_gradient': 4,
            'L0A_phase:dtheta0_deg': 7,
            'L0A_scale:voltage': 6,
            'L0B_phase:dtheta0_deg': 9,
            'L0B_scale:voltage': 8,
            'QA01:b1_gradient': 10,
            'QA02:b1_gradient': 11,
            'QE01:b1_gradient': 12,
            'QE02:b1_gradient': 13,
            'QE03:b1_gradient': 14,
            'QE04:b1_gradient': 15,
            'SOL1:solenoid_field_scale': 3,
            'SQ01:b1_gradient': 5,
            'distgen:r_dist:sigma_xy:value': 0,
            'distgen:t_dist:length:value': 1,
            'distgen:total_charge:value': 2},
 'loc_out': {'norm_emit_x': 3,
             'norm_emit_y': 4,
             'sigma_x': 0,
             'sigma_y': 1,
             'sigma_z': 2},
 'model_in_list': ['distgen:r_dist:sigma_xy:value',
                   'distgen:t_dist:length:value',
                   'distgen:total_charge:value',
                   'SOL1:solenoid_field_scale',
                   'CQ01:b1_gradien

## Demonstrate NN model

In [2]:
from transformers import create_sim_to_nn_transformers
from transformed_model import TransformedModel
# get transformers for normailzation into NN
nn_input_transformer, nn_output_transformer = create_sim_to_nn_transformers(
    "configs/normalization.json"
)
test_min = torch.tensor(model_info["train_input_mins"]).unsqueeze(0)
test_max = torch.tensor(model_info["train_input_maxs"]).unsqueeze(0)

model = torch.load("torch_model.pt").double()

# define the NN surrogate that contains the NN, the input/output transformers for
# simulation units
surrogate = TransformedModel(
    model,
    nn_input_transformer,
    nn_output_transformer
)


In [3]:
import numpy as np
raw_x_data = np.load("data/x_raw_small.npy", allow_pickle=True)
raw_y_data = np.load("data/y_raw_small.npy", allow_pickle=True)

preds = surrogate(torch.tensor(raw_x_data).double())

print(raw_x_data[0])
print(raw_y_data[0])

[ 3.92211981e-01  5.83776995e+00  2.50000000e+02  2.29980597e-01
  3.92231489e-04 -6.47414040e-03  5.80000000e+07  2.79354164e+00
  7.00000000e+07 -9.86897001e+00  2.14861067e+00 -2.41573625e+00
  2.40720925e+00  7.18128851e-01 -3.74875980e+00  2.33949758e+00]
[0.00016864793055100324 0.0015950831162394174 0.0005424259007070327
 1.3960860485109939e-06 1.1762668344852966e-06]


In [ ]:
import matplotlib.pyplot as plt
fig,ax = plt.subplots()